In [1]:
from sklearnex import patch_sklearn
patch_sklearn()
import os
from time import time
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score,accuracy_score,confusion_matrix
from collections import defaultdict
import pickle
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
import matplotlib.pyplot as plt
def plotImage(image,label):
    image = image.reshape((3, 32, 32)).transpose([1, 2, 0])
    plt.imshow(image)
    plt.show()
    print('Predicted:',label)
    

In [3]:
df=pd.read_csv('LetterRec/letter-recognition.csv')
df

,letter,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,D,2,2,3,3,2,7,7,7,6,6,6,4,2,8,3,7
19996,C,7,10,8,8,4,4,8,6,9,12,9,13,2,9,3,7
19997,T,6,9,6,7,5,6,11,3,7,11,9,5,2,12,2,4
19998,S,2,3,4,2,1,8,7,2,6,10,6,8,1,9,5,8


In [4]:
df.describe()

,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
count,20000.000000,20000.000000,20000.000000,20000.00000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.00000,20000.000000,20000.000000,20000.000000,20000.000000,20000.00000
mean,4.023550,7.035500,5.121850,5.37245,3.505850,6.897600,7.500450,4.628600,5.178650,8.282050,6.45400,7.929000,3.046100,8.338850,3.691750,7.80120
std,1.913212,3.304555,2.014573,2.26139,2.190458,2.026035,2.325354,2.699968,2.380823,2.488475,2.63107,2.080619,2.332541,1.546722,2.567073,1.61747
min,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,3.000000,5.000000,4.000000,4.00000,2.000000,6.000000,6.000000,3.000000,4.000000,7.000000,5.00000,7.000000,1.000000,8.000000,2.000000,7.00000
50%,4.000000,7.000000,5.000000,6.00000,3.000000,7.000000,7.000000,4.000000,5.000000,8.000000,6.00000,8.000000,3.000000,8.000000,3.000000,8.00000
75%,5.000000,9.000000,6.000000,7.00000,5.000000,8.000000,9.000000,6.000000,7.000000,10.000000,8.00000,9.000000,4.000000,9.000000,5.000000,9.00000
max,15.000000,15.000000,15.000000,15.00000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.00000,15.000000,15.000000,15.000000,15.000000,15.00000


In [5]:
target=df['letter']
features=df.drop(['letter'],axis=1)

In [33]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.4, random_state=42)

In [11]:
def process(index,k,c,g,models,limit=12000,data=None):
    
    X_train, X_test, y_train, y_test = data
    
    time_start=time()
    index+=1
    
    X_train_limit=X_train[:limit]
    y_train_limit=y_train[:limit]

    print('Kernel:',k,'C:',c,'gamma:',g,'index:',index)
    
    svc=SVC(kernel=k,C=c,gamma=g)
    svc.fit(X_train_limit,y_train_limit)
    
    f1=f1_score(y_test,svc.predict(X_test),average='macro')
    train_acccuracy=svc.score(X_train_limit,y_train_limit)
    test_accuracy=svc.score(X_test,y_test)
    
    time_end=time()
    print('Time Taken:',(time_end-time_start)/60,'minutes')
    models.append([svc,k,c,g,train_acccuracy,test_accuracy,f1,(time_end-time_start)/60])
    return index

In [12]:
kernel=['linear','rbf','poly']
C=[0.1,1,10,100]
gamma=['auto',0.001,0.0001,0.1]
models=[]
index=0
limit = 2000
data=[X_train, X_test, y_train, y_test]
for k in kernel:
    for c in C:
        if(k=='linear'):
            index=process(index,k,c,'auto',models,limit,data)
        else:
            for g in gamma:
                index=process(index,k,c,g,models,limit,data)


Kernel: linear C: 0.1 gamma: auto index: 1
Time Taken: 0.009899898370107015 minutes
Kernel: linear C: 1 gamma: auto index: 2
Time Taken: 0.00956662893295288 minutes
Kernel: linear C: 10 gamma: auto index: 3
Time Taken: 0.011349944273630778 minutes
Kernel: linear C: 100 gamma: auto index: 4
Time Taken: 0.03230007886886597 minutes
Kernel: rbf C: 0.1 gamma: auto index: 5
Time Taken: 0.02524999777475993 minutes
Kernel: rbf C: 0.1 gamma: 0.001 index: 6
Time Taken: 0.02263329823811849 minutes
Kernel: rbf C: 0.1 gamma: 0.0001 index: 7
Time Taken: 0.023533328374226888 minutes
Kernel: rbf C: 0.1 gamma: 0.1 index: 8
Time Taken: 0.026266658306121828 minutes
Kernel: rbf C: 1 gamma: auto index: 9
Time Taken: 0.02100005547205607 minutes
Kernel: rbf C: 1 gamma: 0.001 index: 10
Time Taken: 0.01784993807474772 minutes
Kernel: rbf C: 1 gamma: 0.0001 index: 11
Time Taken: 0.025550039609273274 minutes
Kernel: rbf C: 1 gamma: 0.1 index: 12
Time Taken: 0.023199931780497233 minutes
Kernel: rbf C: 10 gamma: a

In [45]:
models.sort(key=lambda x: x[5],reverse=True)
from tabulate import tabulate
print(tabulate(models, headers=['Kernel', 'C','gamma','Train Accuracy','Test Accuracy','F1 Score','Time Taken']))

Kernel        C  gamma      Train Accuracy    Test Accuracy    F1 Score    Time Taken
--------  -----  -------  ----------------  ---------------  ----------  ------------
rbf        10    auto               1              0.88575   0.885288      0.0223
rbf       100    auto               1              0.88575   0.885288      0.0237332
rbf         1    auto               0.993          0.87175   0.871883      0.0206833
rbf        10    0.1                1              0.861625  0.863619      0.0294833
rbf       100    0.1                1              0.861625  0.863619      0.0303999
rbf         1    0.1                0.9985         0.8535    0.8561        0.0273166
poly        0.1  auto               1              0.8545    0.852759      0.00948331
poly        0.1  0.1                1              0.8545    0.852759      0.0104
poly        1    auto               1              0.8545    0.852759      0.0117834
poly        1    0.1                1              0.8545    0.85275

Best 3 based on kernel Sorted by FScore

2000 of 12000
Kernel    |    C | gamma   |   Train Accuracy |   Test Accuracy  |  F1 Score |   Time Taken|
-------- | ----- | -------|  ----------------  |--------------- | ----------|  ------------|
rbf      |  10   | auto   |            1       |       0.88575  | 0.885288  |     0.0223|
poly     |   0.1 | 0.1    |            1       |       0.854375 | 0.852639  |     0.0237332|
linear   |   0.1 | auto   |            0.8775   |      0.80375  | 0.802889  |     0.0206833|

In [55]:
models.sort(key=lambda x: x[5],reverse=True)

y_pred=models[0][0].predict(X_test)
right_dess=None
false_dess=None

y_tester=[x for x in y_test]
x_tester=[x for x in X_test]
for i in range(len(y_tester)):
    if y_tester[i] is y_pred[i]:
        right_dess=i
    else:
        false_dess=i
    if(right_dess is not None and false_dess is not None):
        break
    

print("False asnwer\n-----------------")
print('Actual Label:',y_tester[false_dess])
print('Predicted Label:',y_pred[false_dess])

print("\nCorrect asnwer:\n-----------------")
print('Actual Label:',y_tester[right_dess])
print('Predicted Label:',y_pred[right_dess])


False asnwer
-----------------
Actual Label: T
Predicted Label: Z

Correct asnwer:
-----------------
Actual Label: L
Predicted Label: L


In [46]:
kernel=['linear','rbf','poly']
C=[0.1,1,10,100]
gamma=['auto',0.001,0.0001,0.1]
models=[]
index=0
limit = 8000

data=[X_train, X_test, y_train, y_test]
for k in kernel:
    for c in C:
        if(k=='linear'):
            index=process(index,k,c,'auto',models,limit,data)
        else:
            for g in gamma:
                index=process(index,k,c,g,models,limit,data)


Kernel: linear C: 0.1 gamma: auto index: 1
Time Taken: 0.04624993403752645 minutes
Kernel: linear C: 1 gamma: auto index: 2
Time Taken: 0.05003335475921631 minutes
Kernel: linear C: 10 gamma: auto index: 3
Time Taken: 0.11750004688898723 minutes
Kernel: linear C: 100 gamma: auto index: 4
Time Taken: 0.6320499499638875 minutes
Kernel: rbf C: 0.1 gamma: auto index: 5
Time Taken: 0.11709992488225301 minutes
Kernel: rbf C: 0.1 gamma: 0.001 index: 6
Time Taken: 0.1146999994913737 minutes
Kernel: rbf C: 0.1 gamma: 0.0001 index: 7
Time Taken: 0.14056668678919473 minutes
Kernel: rbf C: 0.1 gamma: 0.1 index: 8
Time Taken: 0.1477333625157674 minutes
Kernel: rbf C: 1 gamma: auto index: 9
Time Taken: 0.09311667680740357 minutes
Kernel: rbf C: 1 gamma: 0.001 index: 10
Time Taken: 0.079449995358785 minutes
Kernel: rbf C: 1 gamma: 0.0001 index: 11
Time Taken: 0.1365500251452128 minutes
Kernel: rbf C: 1 gamma: 0.1 index: 12
Time Taken: 0.10926669438680013 minutes
Kernel: rbf C: 10 gamma: auto index: 1

In [47]:
models.sort(key=lambda x: x[5],reverse=True)
from tabulate import tabulate
print(tabulate(models, headers=['Kernel', 'C','gamma','Train Accuracy','Test Accuracy','F1 Score','Time Taken']))

Kernel        C  gamma      Train Accuracy    Test Accuracy    F1 Score    Time Taken
--------  -----  -------  ----------------  ---------------  ----------  ------------
rbf       100    auto             1                0.96275   0.962294       0.11026
rbf        10    auto             1                0.96275   0.962292       0.0860672
rbf         1    auto             0.994625         0.954375  0.953699       0.0931167
rbf        10    0.1              1                0.952875  0.952443       0.107383
rbf       100    0.1              1                0.952875  0.952443       0.112983
rbf         1    0.1              0.99825          0.947875  0.947379       0.109267
poly        0.1  auto             0.999875         0.92475   0.924075       0.05095
poly        0.1  0.1              1                0.924625  0.923938       0.0532117
poly        1    auto             1                0.924625  0.923938       0.0545002
poly        1    0.1              1                0.924625  

Best 3 based on kernel Sorted by FScore

8000 of 12000

Kernel  |      C | gamma   |   Train Accuracy  |  Test Accuracy |   F1 Score |   Time Taken|
-------- | ----- | ------- | ---------------- | --------------- | ---------- | ------------|
rbf      | 100   | auto   |          1         |       0.96275 |  0.962294   |    0.11026|
poly     |   0.1 | auto    |         0.999875   |      0.92475 |  0.924075    |   0.0860672|
linear   |   1   | auto    |         0.889       |     0.845875 | 0.84401     |   0.0931167|

In [9]:
kernel=['linear','rbf','poly']
C=[0.1,1,10,100]
gamma=['auto',0.001,0.0001,0.1]
models=[]
index=0
limit = 12000
data=[X_train, X_test, y_train, y_test]

for k in kernel:
    for c in C:
        if(k=='linear'):
            index=process(index,k,c,'auto',models,limit,data)
        else:
            for g in gamma:
                index=process(index,k,c,g,models,limit,data)


Kernel: linear C: 0.1 gamma: auto index: 1
Time Taken: 0.07195034424463907 minutes
Kernel: linear C: 1 gamma: auto index: 2
Time Taken: 0.07781667709350586 minutes
Kernel: linear C: 10 gamma: auto index: 3
Time Taken: 0.16729088226954142 minutes
Kernel: linear C: 100 gamma: auto index: 4
Time Taken: 1.1698525309562684 minutes
Kernel: rbf C: 0.1 gamma: auto index: 5
Time Taken: 0.2665393034617106 minutes
Kernel: rbf C: 0.1 gamma: 0.001 index: 6
Time Taken: 0.3118023117383321 minutes
Kernel: rbf C: 0.1 gamma: 0.0001 index: 7
Time Taken: 0.2593243956565857 minutes
Kernel: rbf C: 0.1 gamma: 0.1 index: 8
Time Taken: 0.4108394026756287 minutes
Kernel: rbf C: 1 gamma: auto index: 9
Time Taken: 0.40554279486338296 minutes
Kernel: rbf C: 1 gamma: 0.001 index: 10
Time Taken: 0.4556949655214945 minutes
Kernel: rbf C: 1 gamma: 0.0001 index: 11
Time Taken: 0.303334641456604 minutes
Kernel: rbf C: 1 gamma: 0.1 index: 12
Time Taken: 0.2541193008422852 minutes
Kernel: rbf C: 10 gamma: auto index: 13
T

In [11]:
models.sort(key=lambda x: x[5],reverse=True)
from tabulate import tabulate
print(tabulate(models, headers=['Kernel', 'C','gamma','Train Accuracy','Test Accuracy','F1 Score','Time Taken']))

Kernel        C  gamma      Train Accuracy    Test Accuracy    F1 Score    Time Taken
--------  -----  -------  ----------------  ---------------  ----------  ------------
rbf        10    auto            1                 0.971     0.970534       0.175049
rbf       100    auto            1                 0.971     0.970534       0.180944
rbf         1    auto            0.994083          0.965625  0.965049       0.405543
rbf        10    0.1             1                 0.964375  0.963821       0.237103
rbf       100    0.1             1                 0.964375  0.963821       0.241554
rbf         1    0.1             0.997833          0.9635    0.962948       0.254119
poly        0.1  auto            0.999417          0.942875  0.942055       0.0753085
poly        1    auto            1                 0.941375  0.940528       0.0893334
poly        1    0.1             1                 0.941375  0.940528       0.0801568
poly       10    auto            1                 0.941375 

Best 3 based on kernel Sorted by FScore

12000 of 12000

Kernel    |    C | gamma   |   Train Accuracy |   Test Accuracy |   F1 Score |   Time Taken|
-------- | ----- | ------- | ---------------- | ---------------|  ---------- | ------------|
rbf      |  10   | auto    |        1         |        0.971    | 0.970534   |    0.175049|
poly      |  0.1 | auto    |        0.999417   |       0.942875 | 0.942055  |    0.0753085|
linear   |   1    |auto    |        0.879333   |       0.853625 | 0.851967    |   0.0778167|

In [50]:
# Nearest Class Centroid
from sklearn.neighbors import NearestCentroid
# Nearest Neighbor
from sklearn.neighbors import KNeighborsClassifier

limit = [2000,8000,12000]
nnc_models=[]
knn_models=[]

for l in limit:    
    X_train_limit=X_train[:l]
    y_train_limit=y_train[:l]
      
    print("Size of limit:",l)
    start=time()
    
    nnc=NearestCentroid()
    nnc.fit(X_train,y_train)
    nnc_models.append([l,accuracy_score(y_train_limit,nnc.predict(X_train_limit)),accuracy_score(y_test,nnc.predict(X_test)),f1_score(y_test,nnc.predict(X_test),average='macro'),time()-start])
    
    
    n_neighbors = [3,5,7,15,17,21]
    for n in n_neighbors:
        knn = KNeighborsClassifier(n_neighbors=n)
        start=time()
        # Fit the model to the data
        knn.fit(X_train, y_train)
        knn_models.append([n,l,accuracy_score(y_train_limit,knn.predict(X_train_limit)),accuracy_score(y_test,knn.predict(X_test)),f1_score(y_test,knn.predict(X_test),average='macro'),time()-start])

Size of limit: 2000
Size of limit: 8000
Size of limit: 12000


In [51]:
knn_models.sort(key=lambda x: x[5],reverse=True)
from tabulate import tabulate
print(tabulate(knn_models, headers=['n_neighbors','Limit','Train Accuracy','Test Accuracy','F1 Score','Time Taken']))

  n_neighbors    Limit    Train Accuracy    Test Accuracy    F1 Score    Time Taken
-------------  -------  ----------------  ---------------  ----------  ------------
           21    12000          0.929667         0.909875    0.909518      0.350971
           17    12000          0.936167         0.92        0.919479      0.324039
           15    12000          0.941917         0.92325     0.922665      0.31899
           21     8000          0.93275          0.909875    0.909518      0.303
           15     8000          0.94275          0.92325     0.922665      0.302964
            5    12000          0.970917         0.940375    0.939854      0.292999
           17     8000          0.938875         0.92        0.919479      0.289002
            7    12000          0.9645           0.936       0.935561      0.288003
            3    12000          0.978167         0.942125    0.941809      0.278991
            5     8000          0.972625         0.940375    0.939854      0.257

In [52]:
nnc_models.sort(key=lambda x: x[4],reverse=True)
print(tabulate(nnc_models, headers=['Limit','Train Accuracy','Test Accuracy','F1 Score','Time Taken']))

  Limit    Train Accuracy    Test Accuracy    F1 Score    Time Taken
-------  ----------------  ---------------  ----------  ------------
  12000          0.577333         0.571125    0.557787     0.0850098
   8000          0.580375         0.571125    0.557787     0.0809999
   2000          0.5755           0.571125    0.557787     0.0809996


In [53]:
for i in range(len(knn_models)):
    print('Difference in F1 Score:',knn_models[i][4]-models[i][5])

Difference in F1 Score: -0.06101560251783089
Difference in F1 Score: -0.051054390998037213
Difference in F1 Score: -0.04238403946854685
Difference in F1 Score: -0.05430300591270909
Difference in F1 Score: -0.04115637757247159
Difference in F1 Score: -0.023093251329626252
Difference in F1 Score: -0.022575781130481087
Difference in F1 Score: -0.004966714124369664
Difference in F1 Score: 0.0012812975386017333
Difference in F1 Score: -0.0006731785640593513
Difference in F1 Score: -0.004966714124369664
Difference in F1 Score: 0.0012812975386017333
Difference in F1 Score: -0.031009328438012518
Difference in F1 Score: -0.02090806844213655
Difference in F1 Score: -0.002810521813054967
Difference in F1 Score: 0.01958003763600158
Difference in F1 Score: 0.059722704882242184
Difference in F1 Score: 0.06892430953504025


In [54]:
for i in range(len(nnc_models)):
    print('Difference in F1 Score:',nnc_models[i][3]-models[i][5])

Difference in F1 Score: -0.4127464701635448
Difference in F1 Score: -0.4127464701635448
Difference in F1 Score: -0.4072615354544983


In [61]:

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
# Encoding the target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(target)

# Normalizing the feature data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.4, random_state=42)


In [62]:
kernel=['linear','rbf','poly']
C=[0.1,1,10,100]
gamma=['auto',0.001,0.0001,0.1]
models=[]
index=0
limit = 12000
data=[X_train, X_test, y_train, y_test]

for k in kernel:
    for c in C:
        if(k=='linear'):
            index=process(index,k,c,'auto',models,limit,data)
        else:
            for g in gamma:
                index=process(index,k,c,g,models,limit,data)


Kernel: linear C: 0.1 gamma: auto index: 1
Time Taken: 0.08010576963424683 minutes
Kernel: linear C: 1 gamma: auto index: 2
Time Taken: 0.06329356034596761 minutes
Kernel: linear C: 10 gamma: auto index: 3
Time Taken: 0.08738329410552978 minutes
Kernel: linear C: 100 gamma: auto index: 4
Time Taken: 0.23948785463968914 minutes
Kernel: rbf C: 0.1 gamma: auto index: 5
Time Taken: 0.13488765954971313 minutes
Kernel: rbf C: 0.1 gamma: 0.001 index: 6
Time Taken: 0.2524568518002828 minutes
Kernel: rbf C: 0.1 gamma: 0.0001 index: 7
Time Taken: 0.20665716330210368 minutes
Kernel: rbf C: 0.1 gamma: 0.1 index: 8
Time Taken: 0.13263335227966308 minutes
Kernel: rbf C: 1 gamma: auto index: 9
Time Taken: 0.07447765668233236 minutes
Kernel: rbf C: 1 gamma: 0.001 index: 10
Time Taken: 0.16280334790547688 minutes
Kernel: rbf C: 1 gamma: 0.0001 index: 11
Time Taken: 0.2523001273473104 minutes
Kernel: rbf C: 1 gamma: 0.1 index: 12
Time Taken: 0.0758563001950582 minutes
Kernel: rbf C: 10 gamma: auto index

In [68]:
models.sort(key=lambda x: x[5],reverse=True)
from tabulate import tabulate
print(tabulate(models, headers=['Kernel', 'C','gamma','Train Accuracy','Test Accuracy','F1 Score','Time Taken']))

Kernel        C  gamma      Train Accuracy    Test Accuracy    F1 Score    Time Taken
--------  -----  -------  ----------------  ---------------  ----------  ------------
rbf        10    0.1             0.99725           0.9675    0.967001       0.0738082
rbf       100    0.1             1                 0.96675   0.966274       0.0665057
rbf        10    auto            0.99325           0.9655    0.964844       0.0719669
rbf       100    auto            0.99975           0.96525   0.964798       0.0654
rbf         1    0.1             0.969333          0.94875   0.948079       0.0758563
poly       10    0.1             0.9935            0.9475    0.947067       0.0688834
poly      100    auto            0.997583          0.94375   0.943277       0.0667831
poly      100    0.1             0.99975           0.942125  0.941588       0.0731968
poly       10    auto            0.978417          0.9405    0.940218       0.0741189
rbf         1    auto            0.952             0.9333

Best 3 based on kernel Sorted by FScore
Using StandarScaler and LabelEncoder
12000 of 12000

Kernel    |    C | gamma  |    Train Accuracy  |  Test Accuracy |   F1 Score  |  Time Taken|
-------- | ----- | -------|  ---------------- | ---------------|  ---------- | ------------|
rbf      |  10  |  0.1    |         0.99725    |       0.9675  |  0.967001   |    0.0738082|
poly     |  10  |  0.1    |         0.9935     |       0.9475  |  0.947067    |   0.0688834|
linear   |   1 |   auto   |         0.8715       |     0.853   |  0.851172     |  0.0632936|

In [63]:
# Nearest Class Centroid
from sklearn.neighbors import NearestCentroid
# Nearest Neighbor
from sklearn.neighbors import KNeighborsClassifier

limit = [2000,8000,12000]
nnc_models=[]
knn_models=[]

for l in limit:    
    X_train_limit=X_train[:l]
    y_train_limit=y_train[:l]
      
    print("Size of limit:",l)
    start=time()
    
    nnc=NearestCentroid()
    nnc.fit(X_train,y_train)
    nnc_models.append([l,accuracy_score(y_train_limit,nnc.predict(X_train_limit)),accuracy_score(y_test,nnc.predict(X_test)),f1_score(y_test,nnc.predict(X_test),average='macro'),time()-start])
    
    
    n_neighbors = [3,5,7,15,17,21]
    for n in n_neighbors:
        knn = KNeighborsClassifier(n_neighbors=n)
        start=time()
        # Fit the model to the data
        knn.fit(X_train, y_train)
        knn_models.append([n,l,accuracy_score(y_train_limit,knn.predict(X_train_limit)),accuracy_score(y_test,knn.predict(X_test)),f1_score(y_test,knn.predict(X_test),average='macro'),time()-start])

Size of limit: 2000
Size of limit: 8000
Size of limit: 12000


In [64]:
knn_models.sort(key=lambda x: x[5],reverse=True)
print(tabulate(knn_models, headers=['n_neighbors','Limit','Train Accuracy','Test Accuracy','F1 Score','Time Taken']))


  n_neighbors    Limit    Train Accuracy    Test Accuracy    F1 Score    Time Taken
-------------  -------  ----------------  ---------------  ----------  ------------
           21    12000          0.925            0.90375     0.903503      0.270998
           17    12000          0.9325           0.911625    0.911094      0.255003
           15    12000          0.93575          0.913375    0.912878      0.245998
           21     8000          0.927            0.90375     0.903503      0.236003
           17     8000          0.933875         0.911625    0.911094      0.217997
            7    12000          0.958            0.9295      0.928964      0.217996
            5    12000          0.965167         0.93275     0.932129      0.216004
            3    12000          0.97575          0.934625    0.934235      0.212998
           15     8000          0.93725          0.913375    0.912878      0.210993
            7     8000          0.9595           0.9295      0.928964      0

In [65]:

nnc_models.sort(key=lambda x: x[4],reverse=True)
print(tabulate(nnc_models, headers=['Limit','Train Accuracy','Test Accuracy','F1 Score','Time Taken']))

  Limit    Train Accuracy    Test Accuracy    F1 Score    Time Taken
-------  ----------------  ---------------  ----------  ------------
   2000          0.5705           0.565625    0.553418     0.0259957
  12000          0.570333         0.565625    0.553418     0.0240009
   8000          0.572375         0.565625    0.553418     0.0220017


In [66]:
for i in range(len(knn_models)):
    print('Difference in F1 Score:',knn_models[i][4]-models[i][5])

Difference in F1 Score: 0.06761310920775121
Difference in F1 Score: 0.059922757846571195
Difference in F1 Score: 0.06373494629404586
Difference in F1 Score: 0.05397612603764079
Difference in F1 Score: 0.08481365541403219
Difference in F1 Score: 0.7743238623666231
Difference in F1 Score: 0.9224750317788647
Difference in F1 Score: 0.08612625176783639
Difference in F1 Score: -0.01976139097760976
Difference in F1 Score: 0.24914241288458072
Difference in F1 Score: 0.7713760881030407
Difference in F1 Score: -0.04457649083294857
Difference in F1 Score: -0.030608941547403345
Difference in F1 Score: 0.09482261633556988
Difference in F1 Score: 0.23244891710984805
Difference in F1 Score: -0.03276570078063934
Difference in F1 Score: -0.03583373827281888
Difference in F1 Score: 0.07400819513995449


In [67]:
for i in range(len(nnc_models)):
    print('Difference in F1 Score:',nnc_models[i][3]-models[i][5])

Difference in F1 Score: -0.28247178603421474
Difference in F1 Score: -0.2977539039562449
Difference in F1 Score: -0.2957249351631427
